In [ ]:
# to read article from MySQL DATABASE: contentanalysis, TABLE: articles
def get_id_from_mysql(sql_database, sql_table, column_name):
    import mysql.connector
    sql_db = mysql.connector.connect(
        host = '127.0.0.1',
        user = 'root',
        password = "",
        database = sql_database
    )
    cursor = sql_db.cursor()
    
    # to check whether TABLE EXISTS OR NOT
    cursor.execute("SHOW TABLES LIKE '{}'".format(sql_table))
    temp_result = cursor.fetchone()
    if (temp_result):
        logging.info("TABLE {} exists in DATABASE {}".format(sql_table, sql_database))
    else:
        logging.info("TABLE {} exists in DATABASE {}".format(sql_table, sql_database))
        
    select_sql = "SELECT {} FROM {}".format(column_name[0], sql_table)
    cursor.execute(select_sql)
    result = cursor.fetchall()
    return result

In [ ]:
def get_mean_article_vector(w2v_model, c_cut, c_id):
    text_words = c_cut.split()
    
    sentence_vec = [0.0]*100
    #print(text_words)
    word_count = 0
    
    # if there is words in text_words,sum up its word_vector as article vector and take its average
    if len(text_words) is not 0:
        for word in text_words:
            try:
                word_vec = w2v_model.wv[word]
                word_count += 1
                sentence_vec = [x+y for x, y in zip(sentence_vec, word_vec)]
            except KeyError:
                logging.info("KeyError Handling: no words {} in {}".format(word, c_id))
                continue
        #length = len(text_words)
        length = word_count
        
        sentence_vec = [round(value/length, 5) for value in sentence_vec]
    else: # if there is no words in the text_words, assign value as -100
        sentence_vec = [-100.0]*100
    #print("length:　" , word_count, len(text_words))
    return sentence_vec

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s', level=logging.INFO)

# to get the cut_corpus each article from the txt file
corpus = []
#with open('advertisement_article_cut.txt', 'r', encoding='utf-8') as content:
with open('advertisement_article_cut_0617.txt', 'r', encoding='utf-8') as content:
    logging.info("start to cut words")
    
    sql_database = "contentanalysis"
    sql_table = "articles"
    column_name = ["id", "content"]
    content_id = get_id_from_mysql(sql_database, sql_table, column_name)  
    
    id_count = 0
    
    for text_num, line_str in enumerate(content):

        line_str = line_str.replace('\n', '')
        line_str = line_str.strip()
        content_list = [content_id[id_count][0], line_str]
        corpus.append(content_list)
        id_count += 1
    print(corpus[:100])
    print(len(corpus))
        
# create c_vecots_df to store all c_vector
import pandas as pd
column_name = ['content_id']
for i in range(100):
    temp_col = 'content_vec_{}'.format(str(i))
    column_name.append(temp_col)
#print(column_name)
c_vectors_df = pd.DataFrame(columns=column_name)

# to load the word2vec model
from gensim.models import word2vec
#model_name = "ad_w2v_model_v1.model"
model_name = "ad_w2v_model_v2_0617.model"
w2v_model = word2vec.Word2Vec.load(model_name)
logging.info("loaded w2v model: {}".format(model_name))

for c_id, c_cut in corpus:
    logging.info("sart create content_vec for {}".format(str(c_id)))
    
    # to get the content vector
    logging.info("start calculate article vector")
    c_vector = get_mean_article_vector(w2v_model, c_cut, c_id)
    logging.info("end calculate article vector")
    
    # to store the c_vector as a total dataframe: c_vectors_df
    logging.info("start combine vectors")
    c_vector.insert(0, c_id)
    c_df = pd.DataFrame([c_vector], columns=c_vectors_df.columns)
    c_vectors_df = pd.concat([c_vectors_df, c_df], ignore_index=True)
    logging.info("end combine vectors")
    logging.info("{} content_vec DONE".format(str(c_id)))
    

# to store the result of content_vec as csv file
c_vectors_df = c_vectors_df.drop_duplicates(subset=['content_id'], ignore_index=True)
#csv_filename = 'content_vectors.csv'
csv_filename = 'content_vectors_0617.csv'
c_vectors_df.to_csv(csv_filename, index=False)
#logging.info("have finished  create content vector and INSERT it INTO TABLE articles_test")
logging.info("have finished  create content vector and output is as {}".format(csv_filename))